In [1]:
#r "nuget: Microsoft.SemanticKernel.Agents.Abstractions, 1.33.0-alpha"
#r "nuget: Microsoft.SemanticKernel.Agents.Core, 1.33.0-alpha"
#r "nuget: Microsoft.SemanticKernel.Agents.OpenAI, 1.17.0-alpha"
#r "nuget: Microsoft.SemanticKernel.Connectors.AzureOpenAI, 1.33.0"


Installed Packages Microsoft.SemanticKernel.Agents.Abstractions, 1.33.0-alpha Microsoft.SemanticKernel.Agents.Core, 1.33.0-alpha Microsoft.SemanticKernel.Agents.OpenAI, 1.17.0-alpha Microsoft.SemanticKernel.Connectors.AzureOpenAI, 1.33.0

In [16]:
using System.ComponentModel;
using Microsoft.SemanticKernel;
using Microsoft.SemanticKernel.Agents;
//using Microsoft.SemanticKernel.Agents.OpenAI;
using Microsoft.SemanticKernel.ChatCompletion;
using Microsoft.SemanticKernel.Connectors.AzureOpenAI;
using Microsoft.SemanticKernel.Agents.Chat;
using Microsoft.SemanticKernel.Connectors.OpenAI;
using Microsoft.SemanticKernel;
using Microsoft.SemanticKernel.ChatCompletion;



In [17]:
var AzureOpenAiEndpoint = "";
var AzureOpenAiKey = "";

In [19]:
Kernel kernel = Kernel.CreateBuilder()
    .AddAzureOpenAIChatCompletion("gpt-4o", AzureOpenAiEndpoint, AzureOpenAiKey)
    .Build();
    

In [20]:
// Lets given the Persona to Agents

string ProgamManager = """
    You are a program manager which will take the requirement and create a plan for creating app. Program Manager understands the 
    user requirements and form the detail documents with requirements and costing. 
""";

string SoftwareEngineer = """
   You are Software Engieer, and your goal is create web app using HTML and JavaScript by taking into consideration all
   the requirements given by Program Manager. 
""";

string ProjectManager = """
    You are manager which will review software engineer code, and make sure all client requirements are completed. 
    You are the guardian of quality, ensuring the final product meets all specifications and receives the green light for release.
     Once all client requirements are completed, you can approve the request by just responding "approve"
""";


In [21]:
#pragma warning disable SKEXP0110, SKEXP0001 // Rethrow to preserve stack details

 ChatCompletionAgent ProgaramManagerAgent =
            new()
            {
                Instructions = ProgamManager,
                Name = "ProgaramManagerAgent",
                Kernel = kernel
            };

 ChatCompletionAgent SoftwareEngineerAgent =
            new()
            {
                Instructions = SoftwareEngineer,
                Name = "SoftwareEngineerAgent",
                Kernel = kernel
            };

 ChatCompletionAgent ProjectManagerAgent =
            new()
            {
                Instructions = ProjectManager,
                Name = "ProjectManagerAgent",
                Kernel = kernel
            };



In [22]:
    #pragma warning disable SKEXP0110, SKEXP0001 // Rethrow to preserve stack details
    
    using System.Threading;
    
    private sealed class ApprovalTerminationStrategy : TerminationStrategy
    {
        // Terminate when the final message contains the term "approve"
        protected override Task<bool> ShouldAgentTerminateAsync(Agent agent, IReadOnlyList<ChatMessageContent> history, CancellationToken cancellationToken)
            => Task.FromResult(history[history.Count - 1].Content?.Contains("approve", StringComparison.OrdinalIgnoreCase) ?? false);
    }

In [23]:
#pragma warning disable SKEXP0110, SKEXP0001 

        AgentGroupChat chat =
            new(ProgaramManagerAgent, SoftwareEngineerAgent, ProjectManagerAgent)
            {
                ExecutionSettings =
                    new()
                    {
                        TerminationStrategy =
                            new ApprovalTerminationStrategy()
                            {
                                Agents = [ProjectManagerAgent],
                                MaximumIterations = 6,
                            }
                    }
            };


In [24]:
        #pragma warning disable SKEXP0110, SKEXP0001 
        // Invoke chat and display messages.
        string input = """
        
        I want to develop app which will provide me calculator. Keep it very simple. And get final approval from manager.
        """;

        chat.AddChatMessage(new ChatMessageContent(AuthorRole.User, input));
        Console.WriteLine($"# {AuthorRole.User}: '{input}'");

        await foreach (var content in chat.InvokeAsync())
        {
            Console.WriteLine($"# {content.Role} - {content.AuthorName ?? "*"}: '{content.Content}'");
        }

# user: '
I want to develop app which will provide me calculator. Keep it very simple. And get final approval from manager.'
# Assistant - ProgaramManagerAgent: 'To develop a simple calculator app, I will outline a basic plan detailing the requirements, features, timeline, and cost estimate for your project. Here is the plan:

### Project Overview
The objective is to develop a straightforward calculator app that performs basic arithmetic operations such as addition, subtraction, multiplication, and division.

### Requirements

#### Functional Requirements
1. **User Interface (UI):**
   - Simple and intuitive user interface.
   - Numeric keypad (0-9) along with basic arithmetic operator buttons (+, -, *, /).
   - A display screen to show input and results.
   - Clear, Equals, and Delete buttons.

2. **Core Features:**
   - Perform addition, subtraction, multiplication, and division operations.
   - Ability to handle multiple operations sequentially.
   - Clear button to reset all inputs

Error: Microsoft.SemanticKernel.HttpOperationException: HTTP 429 (429)

Requests to the ChatCompletions_Create Operation under Azure OpenAI API version 2024-10-01-preview have exceeded token rate limit of your current OpenAI S0 pricing tier. Please retry after 47 seconds. Please go here: https://aka.ms/oai/quotaincrease if you would like to further increase the default rate limit.
 ---> System.ClientModel.ClientResultException: HTTP 429 (429)

Requests to the ChatCompletions_Create Operation under Azure OpenAI API version 2024-10-01-preview have exceeded token rate limit of your current OpenAI S0 pricing tier. Please retry after 47 seconds. Please go here: https://aka.ms/oai/quotaincrease if you would like to further increase the default rate limit.
   at Azure.AI.OpenAI.ClientPipelineExtensions.ProcessMessageAsync(ClientPipeline pipeline, PipelineMessage message, RequestOptions options)
   at Azure.AI.OpenAI.Chat.AzureChatClient.CompleteChatAsync(BinaryContent content, RequestOptions 